<a href="https://colab.research.google.com/github/jlopetegui98/NER-ClinicalTrials-Elegibility-Criteria/blob/main/Roberta%2BLLM/evaluate_roberta_chia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# uncomment if working in colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# uncomment if using colab
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U datasets
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install seqeval
!pip install -q -U evaluate

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=b571c9c4836027705ada02b905790e6d0b00dff2c033b522f11aeb9c3d0d66ed
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built

In [2]:
import numpy as np
from transformers import AutoTokenizer, AutoModelForTokenClassification, DataCollatorForTokenClassification,  Trainer, TrainingArguments
from datasets import load_dataset, load_metric
from seqeval.metrics import classification_report
from seqeval.scheme import IOB2
import evaluate
import torch
from eval_file import *
from utils import *

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
# dict for the entities (entity to int value)
simple_ent = {"Condition", "Value", "Drug", "Procedure", "Measurement", "Temporal", "Observation", "Person", "Device"}
sel_ent = {
    "O": 0,
    "B-Condition": 1,
    "I-Condition": 2,
    "B-Value": 3,
    "I-Value": 4,
    "B-Drug": 5,
    "I-Drug": 6,
    "B-Procedure": 7,
    "I-Procedure": 8,
    "B-Measurement": 9,
    "I-Measurement": 10,
    "B-Temporal": 11,
    "I-Temporal": 12,
    "B-Observation": 13,
    "I-Observation": 14,
    "B-Person": 15,
    "I-Person": 16,
    "B-Device": 17,
    "I-Device": 18
}

entities_list = list(sel_ent.keys())
sel_ent_inv = {v: k for k, v in sel_ent.items()}

In [5]:
root = '..'
root = './drive/MyDrive/TER-LISN-2024'
data_path = f'{root}/data'
models_path = f'{root}/models'

In [6]:
model_name = "roberta-base"

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [8]:
# tokenize and align the labels in the dataset
def tokenize_and_align_labels(sentence, tokenizer, flag = 'I'):
    """
    Tokenize the sentence and align the labels
    inputs:
        sentence: dict, the sentence from the dataset
        flag: str, the flag to indicate how to deal with the labels for subwords
            - 'I': use the label of the first subword for all subwords but as intermediate (I-ENT)
            - 'B': use the label of the first subword for all subwords as beginning (B-ENT)
            - None: use -100 for subwords
    outputs:
        tokenized_sentence: dict, the tokenized sentence now with a field for the labels
    """
    tokenized_sentence = tokenizer(sentence['tokens'], is_split_into_words=True, truncation=True, padding='max_length', max_length=512)

    labels = []
    all_word_ids = []
    for i, labels_s in enumerate(sentence['ner_tags']):
        word_ids = tokenized_sentence.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # if the word_idx is None, assign -100
            if word_idx is None:
                label_ids.append(-100)
            # if it is a new word, assign the corresponding label
            elif word_idx != previous_word_idx:
                label_ids.append(labels_s[word_idx])
            # if it is the same word, check the flag to assign
            else:
                if flag == 'I':
                    if entities_list[labels_s[word_idx]].startswith('I'):
                      label_ids.append(labels_s[word_idx])
                    else:
                      label_ids.append(labels_s[word_idx] + 1)
                elif flag == 'B':
                    label_ids.append(labels_s[word_idx])
                elif flag == None:
                    label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
        all_word_ids.append(word_ids)
    tokenized_sentence['labels'] = labels
    tokenized_sentence['word_ids'] = all_word_ids
    return tokenized_sentence

In [9]:
dataset = load_dataset('JavierLopetegui/chia_v1')

In [10]:
# tokenize and align the labels in the dataset
dataset = dataset.map(lambda x: tokenize_and_align_labels(x, tokenizer, 'I'), batched = True)

Map:   0%|          | 0/1307 [00:00<?, ? examples/s]

In [11]:
dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'file', 'index', 'input_ids', 'attention_mask', 'labels', 'word_ids'],
        num_rows: 8881
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'file', 'index', 'input_ids', 'attention_mask', 'labels', 'word_ids'],
        num_rows: 1307
    })
    val: Dataset({
        features: ['tokens', 'ner_tags', 'file', 'index', 'input_ids', 'attention_mask', 'labels', 'word_ids'],
        num_rows: 2221
    })
})

In [12]:
# load model
model = torch.load(f'{models_path}/roberta-ner-chia.pt')

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [14]:
data_for_model = dataset['test'].remove_columns(['file', 'tokens', 'labels', 'index', 'ner_tags', 'word_ids'])

In [15]:
data_for_model

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 1307
})

In [16]:
data_loader = torch.utils.data.DataLoader(data_for_model, batch_size=8)

In [17]:
model.to(device)

RobertaForTokenClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (L

In [18]:
from tqdm import tqdm

In [19]:
len(data_loader.dataset[2]['attention_mask'])

512

In [20]:
labels = []
for batch in tqdm(data_loader):

    batch['input_ids'] = torch.LongTensor(np.column_stack(np.array(batch['input_ids']))).to(device)
    batch['attention_mask'] = torch.LongTensor(np.column_stack(np.array(batch['attention_mask']))).to(device)
    batch_tokenizer = {'input_ids': batch['input_ids'], 'attention_mask': batch['attention_mask']}
    # break
    with torch.no_grad():
        outputs = model(**batch_tokenizer)

    labels_batch = torch.argmax(outputs.logits, dim=2).to('cpu').numpy()
    labels.extend([list(labels_batch[i]) for i in range(labels_batch.shape[0])])

    del batch
    del outputs
    torch.cuda.empty_cache()

100%|██████████| 164/164 [00:50<00:00,  3.22it/s]


In [21]:
def annotate_sentences(dataset, labels, entities_list,criteria = 'first_label'):
    """
    Annotate the sentences with the predicted labels
    inputs:
        dataset: dataset, dataset with the sentences
        labels: list, list of labels
        entities_list: list, list of entities
        criteria: str, criteria to use to select the label when the words pices have different labels
            - first_label: select the first label
            - majority: select the label with the majority
    outputs:
        annotated_sentences: list, list of annotated sentences
    """
    annotated_sentences = []
    for i in range(len(dataset)):
        # get just the tokens different from None
        sentence = dataset[i]
        word_ids = sentence['word_ids']
        sentence_labels = labels[i]
        annotated_sentence = [[] for _ in range(len(dataset[i]['tokens']))]
        for word_id, label in zip(word_ids, sentence_labels):
            if word_id is not None:
                annotated_sentence[word_id].append(label)
        annotated_sentence_filtered = []
        if criteria == 'first_label':
            annotated_sentence_filtered = [annotated_sentence[i][0] for i in range(len(annotated_sentence))]
        elif criteria == 'majority':
            annotated_sentence_filtered = []
            for j in range(len(annotated_sentence)):
                starts_flag = entities_list[annotated_sentence[j][0]].startswith('B')

                ent = max(set(annotated_sentence[j]), key=annotated_sentence[j].count)
                if starts_flag and ent != 0:
                    label = entities_list[ent][2:]
                    label = 'B-' + label
                    annotated_sentence_filtered.append(sel_ent[label])
                else:
                    annotated_sentence_filtered.append(ent)
        annotated_sentences.append(annotated_sentence_filtered)
    return annotated_sentences

In [ ]:
del model

In [22]:
annotated_sentences_first = annotate_sentences(dataset['test'], labels, entities_list, criteria='first_label')
annotated_sentences_max = annotate_sentences(dataset['test'], labels, entities_list, criteria='majority')

In [ ]:
#load seqeval metric for evaluation
metric = load_metric("seqeval")

<ipython-input-38-653dc96d1cff>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [23]:
def get_labels(p):
    predictions, labels = p
    # Remove ignored index (special tokens)
    predictions = [
        [entities_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    labels = [
        [entities_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    return predictions, labels






In [24]:
pred_labels, true_labels = get_labels((annotated_sentences_first, dataset['test']['ner_tags']))

**Evaluating MLM outputs**

In [33]:
evaluator = BioEval()

In [34]:
evaluator.evaluate_annotations(true_labels, pred_labels, do_lower=True)

In [38]:
evaluator.performance

{'overall': {'acc': 0.8351,
  'strict': {'precision': 0.6225968648328897,
   'recall': 0.6740313800832533,
   'f_score': 0.6472939729397292},
  'relax': {'precision': 0.7580597456373854,
   'recall': 0.8206852385526737,
   'f_score': 0.7881303813038131}},
 'category': {'strict': {'condition': {'precision': 0.6648394675019577,
    'recall': 0.7683257918552037,
    'f_score': 0.7128463476070528},
   'person': {'precision': 0.7133757961783439,
    'recall': 0.8296296296296296,
    'f_score': 0.7671232876712328},
   'value': {'precision': 0.7067039106145251,
    'recall': 0.7207977207977208,
    'f_score': 0.7136812411847672},
   'drug': {'precision': 0.7180043383947939,
    'recall': 0.7471783295711061,
    'f_score': 0.7323008849557522},
   'temporal': {'precision': 0.49279538904899134,
    'recall': 0.5757575757575758,
    'f_score': 0.5310559006211181},
   'measurement': {'precision': 0.5473372781065089,
    'recall': 0.6379310344827587,
    'f_score': 0.5891719745222931},
   'procedur

In [19]:
def get_annotated_sentence(tokens, labels):
    """
    Get the annotated sentence
    inputs:
        tokens: list, list of tokens
        labels: list, list of labels
    outputs:
        annotated_sentence: str, the annotated sentence
    """
    annotated_sentence = []
    i = 0
    sentence = ""
    while i < len(tokens):
        if labels[i] != 'O':
            start = i
            while i < len(tokens) and labels[i] != 'O' and labels[i].split('-')[1] == labels[start].split('-')[1]:
                i += 1
            end = i
            sentence += f"<{labels[start].split('-')[1]}>" + ' '.join(tokens[start:end]) + f"</{labels[start].split('-')[1]}>"
            if end < len(tokens):
                sentence += ' '
        else:
            sentence += tokens[i]
            i += 1
            if i < len(tokens):
                sentence += ' '
    return sentence

In [20]:
get_annotated_sentence(['The', 'patient', 'has', 'a', 'fever', 'of', '38', 'degrees', 'Celsius', '.'], ['O', 'O', 'O', 'O', 'B-Condition', 'O', 'B-Value', 'I-Value', 'I-Value', 'O'])

'The patient has a <Condition>fever</Condition> of <Value>38 degrees Celsius</Value> .'

In [26]:
def get_verification_prompt(sentence):
    """
    Get the verification prompt
    inputs:
        sentence: str, the sentence
    outputs:
        verification_prompt: str, the verification prompt
    """
    return f"""I am working in a named entity recognition task for Clinical trial
eligibility criteria. I have some annotations I need you to check. The list of possible entities is {','.join(entities_list)}.
I want you to check the correcteness of the sentence and return the correct annotations in the exact same format as the original annotations.
Please use just the entities in the list. I will show you a first simple example for you to understand the task:
input: The patient has a <Condition>fever</Condition> of 38 degrees Celsius .
output: The patient has a <Condition>fever</Condition> of <Value>38 degrees Celsius</Value> .

Now it is your turn. Please check the following sentence:
input: {sentence}
output: """

In [27]:
print(get_verification_prompt('The patient has a <Condition>fever</Condition> of 38 degrees Celsius .'))

I am working in a named entity recognition task for Clinical trial
eligibility criteria. I have some annotations I need you to check. The list of possible entities is O,B-Condition,I-Condition,B-Value,I-Value,B-Drug,I-Drug,B-Procedure,I-Procedure,B-Measurement,I-Measurement,B-Temporal,I-Temporal,B-Observation,I-Observation,B-Person,I-Person,B-Device,I-Device.
I want you to check the correcteness of the sentence and return the correct annotations in the exact same format as the original annotations.
Please use just the entities in the list. I will show you a first simple example for you to understand the task:
input: The patient has a <Condition>fever</Condition> of 38 degrees Celsius .
output: The patient has a <Condition>fever</Condition> of <Value>38 degrees Celsius</Value> .

Now it is your turn. Please check the following sentence:
input: The patient has a <Condition>fever</Condition> of 38 degrees Celsius .
output: 


In [29]:
verification_prompts = [get_verification_prompt(get_annotated_sentence(dataset['test'][i]['tokens'], annotated_sentences_first[i])) for i in range(len(dataset['test']))]

**load mistral-7b**

In [ ]:
model_name = "mistralai/Mistral-7B-v0.1"

In [ ]:
# Load base model(Mistral 7B)
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)
model = AutoModelForCausalLM.from_pretrained(
   model_name,
    quantization_config=bnb_config,
    device_map={"": 0}
)

In [ ]:
# import tokenizer for mistral-7B
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.padding_side = 'left'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

In [ ]:
pipe  = TextGenerationPipeline(model = model, tokenizer = tokenizer)

In [ ]:
corrected_sentences = []
for sentence in tqdm(verification_prompts):
    output = pipe(sentence, max_new_tokens = 500, return_full_text = False, handle_long_generation = "hole")[0]['generated_text']
    output = output.split('\n')[0]
    corrected_sentences.append(output)

In [ ]:
len(corrected_sentences)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')

In [ ]:
# tokenize and align the labels in the dataset
def tokenize_and_align_labels(sentence, flag = 'I'):
    """
    Tokenize the sentence and align the labels
    inputs:
        sentence: dict, the sentence from the dataset
        flag: str, the flag to indicate how to deal with the labels for subwords
            - 'I': use the label of the first subword for all subwords but as intermediate (I-ENT)
            - 'B': use the label of the first subword for all subwords as beginning (B-ENT)
            - None: use -100 for subwords
    outputs:
        tokenized_sentence: dict, the tokenized sentence now with a field for the labels
    """
    tokenized_sentence = tokenizer(sentence['tokens'], is_split_into_words=True, truncation=True)

    labels = []
    for i, labels_s in enumerate(sentence['ner_tags']):
        word_ids = tokenized_sentence.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # if the word_idx is None, assign -100
            if word_idx is None:
                label_ids.append(-100)
            # if it is a new word, assign the corresponding label
            elif word_idx != previous_word_idx:
                label_ids.append(labels_s[word_idx])
            # if it is the same word, check the flag to assign
            else:
                if flag == 'I':
                    if entities_list[labels_s[word_idx]].startswith('I'):
                      label_ids.append(labels_s[word_idx])
                    else:
                      label_ids.append(labels_s[word_idx] + 1)
                elif flag == 'B':
                    label_ids.append(labels_s[word_idx])
                elif flag == None:
                    label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_sentence['labels'] = labels
    return tokenized_sentence

In [ ]:
def parse_ann2bio(sentence, pattern, pattern1, pattern2):
    # if sentence[-1] == "\n":
    #     sentence = sentence[:-2] # remove the \n and a final point wrongly added
    # else:
    #     sentence = sentence[:-1] # remove the final point wrongly added
    
    # find the entities
    occurrences = re.finditer(pattern, sentence)
    indexes = [(match.start(), match.end()) for match in occurrences]

    annotation = []
    i = 0


    # create the bio list
    for beg, end in indexes:
        if beg > i:
            annotation.extend([(word, "O") for word in sentence[i:beg].split()])
        entity = sentence[beg:end]
        entity_name = re.search(pattern1, entity).group(1)
        entity = entity.replace(f'<{entity_name}>', "").replace(f'</{entity_name}>', "")
        split_entity = entity.split()
        annotation.append((split_entity[0], "B-" + entity_name))
        annotation.extend([(word, "I-" + entity_name) for word in split_entity[1:]])
        i = end
    annotation.extend([(word, "O") for word in sentence[i:].split()])

    # check punctuation sign in tokens and put them as individual tokens
    ps = r'(\.|\,|\:|\;|\!|\?|\-|\(|\)|\[|\]|\{|\}|\")'
    new_annotation = []
    for i,(word, tag) in enumerate(annotation):
        if re.search(ps, word):
            # find the ocurrences of the punctuation signs
            occurrences = re.finditer(ps, word)
            indexes = [(match.start(), match.end()) for match in occurrences]
            # create the new tokens
            last = 0
            for j, (beg, end) in enumerate(indexes):
                if beg > last:
                    new_annotation.append((word[last:beg], tag))
                if tag != "O":
                    label = f'I-{tag.split("-")[1]}'
                else:
                    label = "O"
                if end < len(word) or (i < len(annotation) - 1 and annotation[i+1][1] == label):
                    new_annotation.append((word[beg:end], label))
                else:
                    new_annotation.append((word[beg:end], 'O')) 
                last = end
            if last < len(word):
                new_annotation.append((word[last:], label))   
                
        else:
            new_annotation.append((word, tag))

    
    return new_annotation

In [ ]:
new_true_annotations = []
for sent in dataset['test']:
    annotation = []
    for word, tag in zip(sent['tokens'], sent['ner_tags']):
        annotation.append((word, entities_list[tag]))
    new_annotation = []
    ps = r'(\.|\,|\:|\;|\!|\?|\-|\(|\)|\[|\]|\{|\}|\")'
    for i,(word, tag) in enumerate(annotation):
        if re.search(ps, word):
            # find the ocurrences of the punctuation signs
            occurrences = re.finditer(ps, word)
            indexes = [(match.start(), match.end()) for match in occurrences]
            # create the new tokens
            last = 0
            for j, (beg, end) in enumerate(indexes):
                if beg > last:
                    new_annotation.append((word[last:beg], tag))
                if tag != "O":
                    label = f'I-{tag.split("-")[1]}'
                else:
                    label = "O"
                if end < len(word) or (i < len(annotation) - 1 and annotation[i+1][1] == label):
                    new_annotation.append((word[beg:end], label))
                else:
                    new_annotation.append((word[beg:end], 'O')) 
                last = end
            if last < len(word):
                new_annotation.append((word[last:], label))
        else:
            new_annotation.append((word, tag))
    new_true_annotations.append(new_annotation)
len(new_true_annotations)

In [ ]:
true_annotations = []
for sent in new_true_annotations:
    dicc_sent = {"tokens":[], "ner_tags":[]}
    for word, tag in sent:
        dicc_sent["tokens"].append(word)
        dicc_sent["ner_tags"].append(sel_ent[tag])
    true_annotations.append(dicc_sent)
len(true_annotations)

In [ ]:
true_df = pd.DataFrame(true_annotations)
true_ann_dataset = Dataset.from_pandas(true_df)

In [ ]:
true_ann_dataset = true_ann_dataset.map(tokenize_and_align_labels, batched=True)

In [ ]:
pattern1 = r'<(Person|Condition|Value|Drug|Procedure|Measurement|Temporal|Observation|Device)>'
pattern2 = r'</(Person|Condition|Value|Drug|Procedure|Measurement|Temporal|Observation|Device)>'
pattern = f'{pattern1}.*?{pattern2}'

In [ ]:
new_corrected_annotations = []
for sent in corrected_sentences:
    annotation = parse_ann2bio(sent, pattern, pattern1, pattern2)
    new_corrected_annotations.append(annotation)
len(new_corrected_annotations)

In [ ]:
corrected_annotations = []
for sent in corrected_annotations:
    dicc_sent = {"tokens":[], "ner_tags":[]}
    for word, tag in sent:
        dicc_sent["tokens"].append(word)
        dicc_sent["ner_tags"].append(sel_ent[tag])
    corrected_annotations.append(dicc_sent)
len(corrected_annotations)

In [ ]:
corrections_df = pd.DataFrame(corrected_annotations)
corrections_dataset = Dataset.from_pandas(corrections_df)

In [ ]:
correctinos_dataset = corrections_dataset.map(tokenize_and_align_labels, batched=True)

In [ ]:
# keep just sentences with the same length
sentences_to_evaluate_corr = []
sentences_to_evaluate_true = []

for i in range(len(corrections_dataset)):
    if len(corrections_dataset['labels'][i]) == len(true_ann_dataset['labels'][i]):
        sentences_to_evaluate_corr.append(corrections_dataset['labels'][i])
        sentences_to_evaluate_true.append(true_ann_dataset['labels'][i])

print(len(sentences_to_evaluate_corr)/len(corrections_dataset))

**Evaluate new annotations**

In [ ]:
evaluator = BioEval()

In [ ]:
pred_labels, true_labels = get_labels((sentences_to_evaluate_corr, sentences_to_evaluate_true))

In [ ]:
evaluator.evaluate_annotations(true_labels, pred_labels, do_lower=True)

In [ ]:
evaluator.performance

In [ ]:
evaluator.save_evaluation('eval_MLM+generative.json')

In [ ]:
evaluator.get_counts()

In [30]:
print('done')

done
